# Automated Machine Learningの使用

モデルのトレーニングに使用できる機械学習アルゴリズムにはさまざまな種類があり、特定のデータおよび予測要件に最も効果的なアルゴリズムを決定するのは容易ではない場合があります。さらに、正規化、欠落した機能の代入などの手法を使用して、トレーニングデータを前処理することにより、モデルの予測パフォーマンスに大きな影響を与えることができます。要件に最適な*モデルを見つけるための探求では、アルゴリズムと前処理変換の多くの組み合わせを試す必要があります。これには多くの時間がかかり、リソースを計算します。

Azure Machine Learningでは、さまざまなアルゴリズムと前処理オプションを使用してトレーニングされたモデルの比較を自動化できます。 [Azure Machine Learning studio](https：//ml/azure.com)のビジュアルインターフェイスまたはSDKを使用して、この機能を活用できます。 SDKを使用すると、自動化された機械学習実験の設定をより細かく制御できますが、視覚的なインターフェイスは使いやすくなっています。このラボでは、SDKを使用したAutomated Machine learningについて学習します。

## ワークスペースに接続する

最初に行う必要があるのは、Azure ML SDKを使用してワークスペースに接続することです。

> **Note**: 前の演習を完了してから、Azureサブスクリプションとの認証済みセッションの有効期限が切れた場合、再認証するように求められます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## Automated Machine Learning用のデータを準備する

Automated Machine Learningのトレーニングスクリプトを作成する必要はありませんが、トレーニングデータを読み込む必要があります。

In [ ]:
import pandas as pd

# Load the data
train_ds = pd.read_csv('data/diabetes.csv')
print('Data ready!')

## Automated Machine Learningを構成する

これで、Automated Machine learning実験を構成する準備が整いました。これを行うには、実験環境に必要なパッケージを含む実行構成、試行する組み合わせの数、モデルを評価するときに使用するメトリックなどを指定する構成設定のセットが必要です。

In [ ]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(name='Automated ML Experiment',
                             task='classification',
                             compute_target='local',
                             training_data = train_ds,
                             n_cross_validations = 2,
                             label_column_name='Diabetic',
                             iterations=6,
                             primary_metric = 'AUC_weighted',
                             max_concurrent_iterations=4,
                             featurization='auto'
                             )

print("Ready for Auto ML run.")

## Automated Machine learning実験を実行する

OK、準備完了です。自動化された機械学習実験を実行しましょう。

> **Note**: これにはかなりの時間がかかります。最初に、コンピューティング環境を準備する必要があります。これが完了すると、実験が開始され、各反復が完了すると進行状況が表示されます。 [Azure Machine Learning studio](https://ml.azure.com)で計算と実験のステータスを監視することもできます.

In [ ]:
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'diabetes_automl')
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion()

## 最高のパフォーマンスモデルを決定する

実験が完了したら、ウィジェットで出力を表示し、最良の結果が得られた実行をクリックして詳細を表示します。
次に、リンクをクリックしてAzureポータルで実験の詳細を表示し、最良の結果をもたらした個々の実行の詳細を表示する前に、実験全体の詳細を表示します。ここには、生成されたモデルのパフォーマンスに関する多くの情報があります。

最高の実行とそれが生成したモデルを取得しましょう。

In [ ]:
best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Automated Machine learningには、[Scikit-Learn Transformationパイプライン]（https://scikit-learn.org/stable/modules/compose.html#combining-estimators）を使用して達成されるデータの前処理を試みるオプションが含まれますAzure Machine Learningパイプラインと混同しないでください！）。これらは、推論の前にデータを変換する手順を含むモデルを生成します。このようなモデルのステップを表示できます:

In [ ]:
for step in fitted_model.named_steps:
    print(step)

最後に、最良のパフォーマンスのモデルを見つけたら、それを登録できます。

In [ ]:
from azureml.core import Model

# Register model
best_run.register_model(model_path='outputs/model.pkl', model_name='diabetes_model_automl',
                        tags={'Training context':'Auto ML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **詳しくは**: Automated Machine learningの詳細については、[Azure MLドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-configure-auto-train)を参照してください。.